# PCA

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split

In [2]:
class PCA:
    def __init__(self, n_components, whiten, random_state=41, gram_iter=100):
        self.n_components = n_components
        self.whiten = whiten
        self.gram_iter = gram_iter
        
    def covariance(self,data): # clear
        mean = np.mean(data,axis = 0)

        centered_data = data - mean

        cov = np.dot(centered_data.T, centered_data) / (centered_data.shape[0] - 1)

        return cov

    def max_off_diagonal(self, matrix):
        # 대각 성분을 제외한 절대값의 최대값을 찾는 함수
        no_diag_matrix = matrix.copy()  # 원본 행렬을 복사

        # 대각 성분을 0으로 설정
        np.fill_diagonal(no_diag_matrix, 0)

        # 대각 성분을 제외한 절대값의 최대값 반환
        return np.max(np.abs(no_diag_matrix))


    def whitening(self, data):
        """데이터를 whiten 변환 (평균 제거 후 표준편차로 나누기)"""
        mean = np.mean(data, axis=0)  # 각 특징의 평균 계산
        std_dev = np.std(data, axis=0)  # 각 특징의 표준편차 계산

        # 표준편차가 0인 경우 1로 설정 (0으로 나누는 것을 방지하기 위해)
        std_dev[std_dev == 0] = 1

        # Whitening 변환 적용
        whitened_data = (data - mean) / std_dev

        return whitened_data

    def gram_schmidt(self, matrix):
        """Gram-Schmidt 과정을 통해 Q와 R을 동시에 구하는 함수"""
        n_vectors = matrix.shape[1]  # 열 벡터의 개수
        Q = np.zeros_like(matrix)  # 직교 기저를 저장할 행렬
        R = np.zeros((n_vectors, n_vectors), dtype=np.float64)  # R 행렬

        for i in range(n_vectors):
            # 처음에는 주어진 벡터를 그대로 사용
            q_i = matrix[:, i]

            # 이전 기저들에 대한 정사영을 빼줌
            for j in range(i):
                q_j = Q[:, j]
                R[j, i] = np.dot(q_j, matrix[:, i])  # R[j, i]를 정사영으로 저장
                q_i -= R[j, i] * q_j  # q_i에서 정사영을 빼줌

            # 직교화된 벡터를 Q 행렬의 i번째 열에 추가
            R[i, i] = np.linalg.norm(q_i)  # R[i, i]는 q_i의 노름
            Q[:, i] = q_i / R[i, i]  # 정규화하여 단위 벡터로 만듦

        return Q, R
    
    def eig(self, data, max_iter=200):
        """QR 알고리즘을 통해 고유값과 고유벡터를 추출하는 함수"""
        # 공분산 행렬 계산
        cov_matrix = self.covariance(data)

        # QR 분해 초기화
        A = cov_matrix.copy()
        n = A.shape[0]
        Q_total = np.eye(n)  # 고유벡터들을 저장할 행렬

        for i in range(max_iter):
            # QR 분해 수행
            Q, R = self.gram_schmidt(A)

            # 행렬 갱신 (R * Q)
            A = np.dot(R, Q)

            # 고유벡터 계산을 위해 Q 행렬을 누적
            Q_total = np.dot(Q_total, Q)

            max_off_diag = self.max_off_diagonal(A)
            print(f"Iteration {i}, 대각 성분 제외 최대값: {max_off_diag}")

        # 대각 성분들이 고유값, Q_total이 고유벡터
        eigenvalues = np.diag(A)
        eigenvectors = Q_total

        return eigenvalues, eigenvectors

    
    def fit(self, data):
        # 1. 공분산 행렬을 기반으로 QR 알고리즘을 사용해 고유값과 고유벡터를 추출
        eigenvalues, eigenvectors = self.eig(data)

        # 2. 고유값을 크기순으로 정렬하여 상위 n개의 고유값과 고유벡터를 선택
        idx = np.argsort(eigenvalues)[::-1] # 고유값을 내림차순으로 정렬
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]

        # 3. 상위 n개의 고유값 및 고유벡터 선택
        self.components_ = eigenvectors[:, :self.n_components]
        self.explained_variance_ = eigenvalues[:self.n_components]

        # 4. Whitening 옵션이 켜져 있을 경우 고유벡터를 whitening 처리
        if self.whiten:
            self.components_ = self.whitening(self.components_)  # whitening 처리

        # 5. 최종적으로 pca_output 계산
        pca_output = np.dot(data, self.components_)

        # 6. PCA 결과를 반환
        return pca_output

    

In [3]:
people = fetch_lfw_people(min_faces_per_person=100, resize=0.3)
image_shape = people.images[0].shape
print("dataset keys        : ", people.keys())
print("dataset.images shape: ", people.images.shape)
print("dataset.data shape  : ", people.data.shape)
print("dataset.target shape: ", people.target.shape)

dataset keys        :  dict_keys(['data', 'images', 'target', 'target_names', 'DESCR'])
dataset.images shape:  (607, 37, 28)
dataset.data shape  :  (607, 1036)
dataset.target shape:  (607,)


In [4]:
pca = PCA(200, False)
n_component_data = pca.fit(people.data)
pca_w = PCA(200, True)
n_component_data_w = pca_w.fit(people.data)
X_train, X_test, y_train, y_test = train_test_split(n_component_data, people.target, test_size=0.25, random_state=41)
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(n_component_data_w, people.target, test_size=0.25, random_state=41)

Iteration 0, 대각 성분 제외 최대값: 1.9492193615110616
Iteration 1, 대각 성분 제외 최대값: 2.6235733292113106
Iteration 2, 대각 성분 제외 최대값: 2.585466796626074
Iteration 3, 대각 성분 제외 최대값: 2.2147283135538696
Iteration 4, 대각 성분 제외 최대값: 2.374682166844347
Iteration 5, 대각 성분 제외 최대값: 3.010636667577677
Iteration 6, 대각 성분 제외 최대값: 2.3358231045344002
Iteration 7, 대각 성분 제외 최대값: 3.290161863594888
Iteration 8, 대각 성분 제외 최대값: 2.6510149455988534
Iteration 9, 대각 성분 제외 최대값: 3.245013202903191
Iteration 10, 대각 성분 제외 최대값: 2.8557918040484243
Iteration 11, 대각 성분 제외 최대값: 3.1430982259138296
Iteration 12, 대각 성분 제외 최대값: 2.9741578953763783
Iteration 13, 대각 성분 제외 최대값: 2.971833025439469
Iteration 14, 대각 성분 제외 최대값: 3.034410720021142
Iteration 15, 대각 성분 제외 최대값: 2.659736664354426
Iteration 16, 대각 성분 제외 최대값: 3.046092180064177
Iteration 17, 대각 성분 제외 최대값: 2.6732023147724924
Iteration 18, 대각 성분 제외 최대값: 3.0176902021568877
Iteration 19, 대각 성분 제외 최대값: 2.7905679829910017
Iteration 20, 대각 성분 제외 최대값: 2.959249828633449
Iteration 21, 대각 성분 제외 최대값: 2.839

# KNN

In [5]:
from sklearn.neighbors import KNeighborsClassifier as knc

In [6]:
knn = knc(n_neighbors=1)
knn_w = knc(n_neighbors=1)

In [7]:
knn.fit(X_train, y_train)
knn_w.fit(X_train_w, y_train_w)

KNeighborsClassifier(n_neighbors=1)

In [8]:
y_pred = knn.predict(X_test)
y_pred_w = knn_w.predict(X_test_w)

c:\Users\wjdgh\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 지정된 파일을 찾을 수 없습니다
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\wjdgh\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\wjdgh\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\wjdgh\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\wjdgh\App

# Results
#### -F1 score를 각 채점 항목 별로 출력하면 됩니다

In [9]:
from sklearn.metrics import f1_score

In [10]:
print(f"Naive PCA: {f1_score(y_test, y_pred)}\nWhitening PCA: {f1_score(y_test_w, y_pred_w)}")

Naive PCA: 0.43902439024390244
Whitening PCA: 0.65
